# LangGraph — Core Concepts & Hands-On

What is LangGraph?
- LangGraph helps you build stateful, multi-step LLM workflows by drawing a little graph in code.

Components in Langraph:

- A State: a shared “notebook” (a Python dictionary) where steps write/read variables.

- Nodes: steps (Python functions) that read the state and return updates.

- Edges: arrows that say which node runs next.

- Entry point: where the graph starts.

- END: where it stops.

- Checkpointing: optional saving of state between runs (so you can resume a conversation).

- Streaming: watch nodes run, step by step.

What we’ll cover:

- Minimal linear graph (2 nodes)

- Conversational agent graph that can call tools (calculator/search)

- Branching with a router (conditional edges)

- Checkpointing + Streaming

- What each LangGraph component is and why it exists

## Bootstrap

⚓--- Before proceeding futher it is very important you do the following: --- 👾

Select the 🗝 (key) icon in the left pane and include your OpenAI Api key with Name as "OPENAPI_KEY" and value as the key, and grant it notebook access in order to be able to run this notebook.

Run the below two cells in the order they are in, before running further cells. Wait till a number appears in place of '*' or '[ ]'. Below the cell you should see "✅ Setup complete".

In [ ]:
pip install -q -U langchain langchain-openai langgraph

In [ ]:
# Environment: load your API key from secrets
from google.colab import userdata

key = userdata.get('OPENAI_API_KEY')  # returns None if not granted
if not key:
    raise RuntimeError("Set OPENAI_API_KEY in a .env file next to this notebook.")

# LangChain (model & messages)
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# LangGraph core pieces
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import MessagesState  # a ready-made "state shape" for chat: {"messages": [...]}
from langgraph.checkpoint.memory import MemorySaver # simple in-memory checkpointing
from langgraph.prebuilt import ToolNode              # a helper node that can execute tools for you

# Tools decorator (to declare a function as a "tool" the model can call)
from langchain_core.tools import tool

print("✅ Setup complete")

## Core Components in LangGraph

### State

Think of State as a shared dictionary.
Example shape: {"topic": "...", "draft": "...", "summary": "..."}.

Every node can read from state and return updates (a small dict) to merge back.

### Node

A Node is a Python function that takes the current state and returns a dict with new or updated keys.

### Edge

An Edge connects one node to the next. You’ll define a start node and how the flow moves, until END.

## Simple Langgraph example

Let's create a Langgraph example where for a given topic a draft is created and then the draft is summarized.

In [ ]:
# A small model to generate text
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, api_key=key)

# 1) Define a "State shape" using a simple Python type comment for clarity:
# We'll store: topic (string), draft (string), summary (string)
# In Python you can just treat the state as a dict with these keys.

# 2) Node 1: create a draft from the topic
def draft_node(state: dict) -> dict:
    topic = state["topic"]
    draft = llm.invoke([("user", f"Write a short (80-120 words) draft about: {topic}")]).content
    return {"draft": draft}

# 3) Node 2: summarize the draft
def summarize_node(state: dict) -> dict:
    draft = state["draft"]
    summary = llm.invoke([("user", f"Summarize in exactly 3 bullets:\n\n{draft}")]).content
    return {"summary": summary}

# 4) Build the graph
g = StateGraph(dict)        # we tell LangGraph "our state is a dict"
g.add_node("draft", draft_node)         # add the first node, give it a name
g.add_node("summarize", summarize_node) # add the second node

g.set_entry_point("draft")              # where the graph starts
g.add_edge("draft", "summarize")        # arrow: draft -> summarize
g.add_edge("summarize", END)            # arrow: summarize -> END (stop)

# 5) Compile the graph into an "app" you can run
app = g.compile()

# 6) Run it once
final_state = app.invoke({"topic": "LangGraph basics"})
print("=== FINAL STATE ===")
for k, v in final_state.items():
    print(f"{k.upper()}:\n{v}\n")

### Langgraph components used

`StateGraph(dict)`: the graph builder. Here, it defines a workflow where state is always a Python dict.

`add_node(name, fn)`: registers a node function.

`set_entry_point(name)`: sets the start node.

`add_edge(from, to)`: connects two nodes (arrows).

`END`: a special label telling the graph to stop.

`compile()`: freezes the builder into an executable app. Produces an app that uses the Runnable interface.

## Conversational Graph with Tools

Let's create a Conversational Chatbot with Messages in the state and have the Agents use tools.

### 1. Define the tools

In [ ]:
@tool
def calculator(expression: str) -> str:
    """Evaluate basic math expressions like '2 + 2 * 5'."""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

@tool
def search_web(query: str) -> str:
    """Fake web search that returns tiny helpful snippets."""
    data = {
        "LangGraph": "LangGraph builds stateful LLM workflows as graphs.",
        "MMR": "Maximal Marginal Relevance balances relevance & diversity."
    }
    return data.get(query, f"No results for '{query}'")

tools = [calculator, search_web]

### Build the agent graph

In [ ]:
llm_tools = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, api_key=key).bind_tools(tools)

def call_model(state: MessagesState) -> dict:
    # state["messages"] is a Python list of messages (HumanMessage/AIMessage/ToolMessage)
    ai_reply = llm_tools.invoke(state["messages"])
    # Return a dict with "messages": [<new message>] so LangGraph appends it
    return {"messages": [ai_reply]}

# Node: "tools" — executes the requested tool(s) and returns ToolMessage(s)
tools_node = ToolNode(tools)

# Router: look at the last AIMessage — did it ask for tools?
def needs_tools_router(state: MessagesState) -> str:
    msgs = state.get("messages", [])
    if not msgs:
        return END
    last = msgs[-1]
    # Some providers put tool calls on .tool_calls, others in .additional_kwargs["tool_calls"]
    if getattr(last, "tool_calls", None):
        return "tools"
    if getattr(last, "additional_kwargs", {}).get("tool_calls"):
        return "tools"
    return END

# Build the graph over MessagesState
ag = StateGraph(MessagesState)
ag.add_node("call_model", call_model)
ag.add_node("tools", tools_node)

ag.set_entry_point("call_model")
# If "call_model" says tools are needed, go to "tools"; otherwise, END
ag.add_conditional_edges("call_model", needs_tools_router, {"tools": "tools", END: END})
# After "tools" run, go back to "call_model" to continue the dialogue
ag.add_edge("tools", "call_model")

# Optional: checkpointing — store conversation per user/session
checkpointer = MemorySaver()
agent_app = ag.compile(checkpointer=checkpointer)

print("✅ Agent graph ready")

### Now let's test it out

by running a multi-turn session (with streaming)

In [ ]:
# Each user/session gets a thread_id for checkpointing
cfg = {"configurable": {"thread_id": "user-001"}}

# Turn 1: user asks something that needs calculator and search
print("=== STREAM (Turn 1) ===")
steps = agent_app.stream({"messages": [HumanMessage(content="Compute 2+2*5, then search LangGraph.")]}, config=cfg)
for step in steps:
    # step is a dict with one key (the node that just ran), e.g. {"call_model": {...}}, {"tools": {...}}
    print(list(step.keys()))

# Turn 2: follow-up — the graph continues from saved state (checkpoint)
print("\n=== STREAM (Turn 2) ===")
steps = agent_app.stream({"messages": [HumanMessage(content="Summarize what you found in 2 bullets.")]}, config=cfg)
for step in steps:
    print(list(step.keys()))

# Inspect the latest messages
curr = agent_app.get_state(cfg)
print("\n=== LAST 6 MESSAGES ===")
for m in curr.values["messages"][-6:]:
    print(m.type.upper()+":", getattr(m, "content", ""))

### How it works?

A lot is happening in this Conversational Graph with tools. It can be a bit overwhelming. Let's go through this one by one.

Since we're building a Conversational Graph with tools. The state will be `MessagesState` which will contain a list of mentions. This is an inbuilt state by Langgraph to allow for the ease of storing messages (HumanMessage, AIMessage, SystemMessage) as the state.

```python
ag = StateGraph(MessagesState)
```



1. `call_model`: runs the LLM against the conversation `(state["messages"])`, and returns the new `AIMessage`.

> You'd be right to think if the Node is returning the state with the same key but with a different value (here, `AIMessage`) the previous value(s) is overwritten, right? That's true but `MessagesState` has a special reducer on the `messages` key called `add_messages`. So, when a node returns `{"messages": [ai_reply]}`, LangGraph merges that into the existing state by extending the list (not replacing it).

2. If the llm thinks we need tools for the user query a `tool_calls` attribute is added to the `AIMessage`.

3. `add_conditional_edges`: In this part, once out of the `call_model`, we are passing the state to the `needs_tools_router`, if the node returns "tools", we are adding an edge to the `tools_node`, if not we're ending the graph there using `END`. This is declared once at the build time.

4. `needs_tools_router`: This checks the last message in the state, `AIMessage` (result of the node `call_model`) if it contains an attribute called `tool_calls`, if so then we return "tools".

5. `tools_node`: Here we use `ToolNode(tools)` a ready-made node that inspects the last `AIMessage` for tool calls, executes them, and produces `ToolMessage`(s). This is appended to the state (messages).







In summary, `call_model` appends a new `AIMessage` to `MessagesState.messages` via the state's inbuilt `add_messages` reducer; if that `AIMessage` includes `tool_calls`, the build-time `add_conditional_edges` sends control (via `needs_tools_router`) to `ToolNode`, which executes the requested tools and appends corresponding `ToolMessages`, then loops back to call_model—and when no `tool_calls` are present, the router directs the graph to `END`.

### Checkpointer and MemorySaver?

Checkpointer:
- A persistence layer that saves and restores the graph state (e.g., MessagesState) between runs/turns so you can resume exactly where you left off (multi-turn, interruptions, retries).
- Enables long-lived agents: In our case Turn 2 doesn’t need the full history; it’s loaded from storage. Instead of storing the state in any backend and retrieving it for subsequent turns and appending it to each turn, this allows to persist the state in-memory and automatically loads the history from the storage.

MemorySaver:
- An in-memory checkpointer—fast and simple for local/dev. For production use Redis using something like langgraph-checkpoint-redis.


**How do you use it?**

Provide a checkpointer at compile time and a `thread_id` at run time:

```python
checkpointer = MemorySaver()
agent_app = ag.compile(checkpointer=checkpointer)
cfg = {"configurable": {"thread_id": "user-001"}}
agent_app.stream({"messages": [HumanMessage(content="...")]}, config=cfg)
```

- `get_state(cfg)` inspects the latest persisted state for the given configuration.

- `thread_id` defines the session identity. If a different value is given, a new conversation with clean state happens, if an existing value is given, state is persisted (In our case, the state is messages, so the thread_id becomes user session).